In [ ]:
mport numpy as np

img = cv.imread('soccer.jpg')	# 영상 읽기
img_show = np.copy(img)			# 붓 칠을 디스플레이할 목적의 영상

mask = np.zeros((img.shape[0], img.shape[1]), np.uint8)
mask[:, :] = cv.GC_PR_BGD		# 모든 화소를 배경일 것 같음으로 초기화

BrushSiz = 9				# 붓의 크기
LColor, RColor = (255,0,0), (0,0,255)	# 파란색(물체)과 빨간색(배경)

painting_flag = False	# 붓 칠하는 중인지 여부

def painting(event, x, y, flags, param):
    global painting_flag

    if event == cv.EVENT_LBUTTONDOWN:  
        cv.circle(img_show, (x, y), BrushSiz, LColor, -1)	# 왼쪽 버튼 클릭하면 파란색
        cv.circle(mask, (x, y), BrushSiz, cv.GC_FGD, -1)
        painting_flag = True
       
    elif event == cv.EVENT_RBUTTONDOWN:
        cv.circle(img_show, (x, y), BrushSiz, RColor, -1)	# 오른쪽 버튼 클릭하면 빨간색
        cv.circle(mask, (x, y), BrushSiz, cv.GC_BGD, -1)
        painting_flag = True
       
    elif event == cv.EVENT_MOUSEMOVE and flags == cv.EVENT_FLAG_LBUTTON:
        if painting_flag:
            cv.circle(img_show, (x, y), BrushSiz, LColor, -1)# 왼쪽 버튼 클릭하고 이동하면 파란색
            cv.circle(mask, (x, y), BrushSiz, cv.GC_FGD, -1)
           
    elif event == cv.EVENT_MOUSEMOVE and flags == cv.EVENT_FLAG_RBUTTON:
        if painting_flag:
            cv.circle(img_show, (x, y), BrushSiz, RColor, -1)	# 오른쪽 버튼 클릭하고 이동하면 빨간색
            cv.circle(mask, (x, y), BrushSiz, cv.GC_BGD, -1)
   
    elif event == cv.EVENT_LBUTTONUP or event == cv.EVENT_RBUTTONUP:
        painting_flag = False
       
    cv.imshow('Painting', img_show)

cv.namedWindow('Painting')
cv.setMouseCallback('Painting', painting)

while(True):
    key = cv.waitKey(1)
   
    if key == ord('q'):
        break
       
    elif key == ord('c'):  # 'c' 키를 누르면 다시 붓칠 가능하도록 초기화
        img_show = np.copy(img)
        mask[:, :] = cv.GC_PR_BGD
        cv.imshow('Painting', img_show)

# 여기부터 GrabCut 적용하는 코드
background=np.zeros((1,65),np.float64)	# 배경 히스토그램 0으로 초기화
foreground=np.zeros((1,65),np.float64)	# 물체 히스토그램 0으로 초기화

cv.grabCut(img,mask,None,background,foreground,5,cv.GC_INIT_WITH_MASK)
mask2=np.where((mask==cv.GC_BGD)|(mask==cv.GC_PR_BGD),0,1).astype('uint8')
grab=img*mask2[:,:,np.newaxis]
cv.imshow('Grab cut image',grab)  

cv.waitKey()
cv.destroyAllWindows()